In [2]:
%cd ..
%pwd

/home/julius/projects/University/2023W/PP2/Project


'/home/julius/projects/University/2023W/PP2/Project'

In [20]:
import yaml
from pathlib import Path

import torch
from tqdm import tqdm

from src.models import FNN, SETHClone

In [ ]:
embedding_dimensions = {"prott5": 1024, "esm2_3b": 2560}
model_classes = {"fnn": FNN, "cnn": SETHClone}

In [36]:
def load_models(run_name: str, model_dir: Path = Path("./models")) -> list[torch.nn.Module]:
    run_dir = model_dir / run_name
    config = yaml.safe_load((run_dir / "config.yml").open())
    embedding_dim = embedding_dimensions[config["data"]["embedding_type"]]
    params = config["model"]["params"]
    model_class = model_classes[config["model"]["type"]]

    map_location = torch.device("cpu") if not torch.cuda.is_available() else None
    return [
        model_class(n_features=embedding_dim, **params).load_state_dict(torch.load(run_dir / f"fold_{i}.pt", map_location=map_location))
        for i in range(config["training"]["n_splits"])
    ]

In [37]:
run_name = "01-19 23:17_strict_prott5_fnn_epoch_1000"
models = load_models(run_name)

In [38]:
models[0]

<All keys matched successfully>